In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import statistics
import time
import datetime

In [2]:
col_name=["userid", "venueid", "catid", "catname", "latitute", "longitude", "timezone", "time"]

In [3]:
#對東京資料過濾，POI和使用者的簽到記錄需要大於10才保留
data = pd.read_csv('../data/foursquare/TKY.txt',sep='\t' , header=None,encoding='latin-1',names=col_name)
print(len(data))
data=data.drop_duplicates()

print('訓練資料集總數:',len(data))
print(data.columns)
print('user : ',len(data.groupby('userid')))
print('poi : ',len(data.groupby('venueid')))
print('cat : ',len(data.groupby('catid')))
print('cat name: ',len(data.groupby('catname')))
filter1= data.groupby('venueid').filter(lambda x: len(x) >= 10)
data= filter1.groupby('userid').filter(lambda x: len(x) >= 10)
print("poi過濾後:",len(filter1),end='\t')
print("user過濾後:",len(data))
print('user : ',len(data.groupby('userid')))
print('poi : ',len(data.groupby('venueid')))
print('cat : ',len(data.groupby('catname')))

573703
訓練資料集總數: 573126
Index(['userid', 'venueid', 'catid', 'catname', 'latitute', 'longitude',
       'timezone', 'time'],
      dtype='object')
user :  2293
poi :  61858
cat :  385
cat name:  247
poi過濾後: 447076	user過濾後: 447076
user :  2293
poi :  7870
cat :  190


In [6]:
#對紐約資料過濾
data2 = pd.read_csv('../data/foursquare/NYC.txt',sep='\t' , header=None,encoding='latin-1',names=col_name)
print(len(data2))
data2=data2.drop_duplicates()

print('訓練資料集總數:',len(data2))
print(data2.columns)
print('user : ',len(data2.groupby('userid')))
print('poi : ',len(data2.groupby('venueid')))
print('cat : ',len(data2.groupby('catid')))
print('cat name: ',len(data2.groupby('catname')))
filter1= data2.groupby('venueid').filter(lambda x: len(x) >= 10)
data2= filter1.groupby('userid').filter(lambda x: len(x) >= 10)
print("poi過濾後:",len(filter1),end='\t')
print("user過濾後:",len(data2))
print('user : ',len(data2.groupby('userid')))
print('poi : ',len(data2.groupby('venueid')))
print('cat : ',len(data2.groupby('catname')))

227428
訓練資料集總數: 227178
Index(['userid', 'venueid', 'catid', 'catname', 'latitute', 'longitude',
       'timezone', 'time'],
      dtype='object')
user :  1083
poi :  38333
cat :  400
cat name:  251
poi過濾後: 147729	user過濾後: 147729
user :  1083
poi :  5130
cat :  208


In [7]:
#處理資料，產生新的特徵
def data_process(data):
    timestamp = []
    hour = []
    day = []
    week = []
    hour_48 = []
    venuedic={}
    catdic={}
    poi=[]
    cat=[]
    poicount=0
    catcount=0
    print(len(data))
    for i in range(len(data)):
        times = data['time'].values[i]
        venue = data['venueid'].values[i]
        categ = data['catname'].values[i]
        timestamp.append(time.mktime(time.strptime(times, '%a %b %d %H:%M:%S %z %Y')))              #總秒數
        t = datetime.datetime.strptime(times,'%a %b %d %H:%M:%S %z %Y')                             #datatime元組
        year = int(t.strftime('%Y'))
        day_i = int(t.strftime('%j'))
        week_i = int(t.strftime('%w'))
        hour_i = int(t.strftime('%H'))
        hour_i_48 = hour_i
        if week_i == 0 or week_i == 6:
            hour_i_48 = hour_i + 24

        if year == 2013:
            day_i = day_i + 366
        
        if venue not in venuedic.keys():
            venuedic[venue]=poicount
            poicount+=1
        if categ not in catdic.keys():
            catdic[categ]=catcount
            catcount+=1

        poi.append(venuedic[venue])
        cat.append(catdic[categ])
        day.append(day_i)
        hour.append(hour_i)
        hour_48.append(int(hour_i_48))
        week.append(week_i)

    data['venueid']=poi
    data['categid']=cat
    data['timestamp'] = timestamp
    data['hour'] = hour
    data['day'] = day
    data['week'] = week
    data['hour_48'] = hour_48

    print(poicount)
    print(catcount)
    return data

In [10]:
data=data_process(data)
data2=data_process(data2)

447076
7870
190
227178
38333
251


In [11]:
data.to_csv('../data/tky_filter_data.csv', index=False)
#data2.to_csv('../data/nyc_filter_data.csv', index=False)
data2.to_csv('../data/nyc_data.csv', index=False)